# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4b508e1ee0>
├── 'a' --> tensor([[ 0.9160, -1.1817, -0.2915],
│                   [ 0.9803, -0.0697,  0.6740]])
└── 'x' --> <FastTreeValue 0x7f4b508e1fd0>
    └── 'c' --> tensor([[ 0.4645, -1.2541, -0.1804, -0.9229],
                        [-1.7030, -0.9680,  2.1924, -0.0436],
                        [-1.1008, -0.3390,  0.7427,  1.2755]])

In [4]:
t.a

tensor([[ 0.9160, -1.1817, -0.2915],
        [ 0.9803, -0.0697,  0.6740]])

In [5]:
%timeit t.a

72.8 ns ± 1.39 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4b508e1ee0>
├── 'a' --> tensor([[1.2090, 3.2932, 0.3761],
│                   [0.1010, 0.8741, 0.5837]])
└── 'x' --> <FastTreeValue 0x7f4b508e1fd0>
    └── 'c' --> tensor([[ 0.4645, -1.2541, -0.1804, -0.9229],
                        [-1.7030, -0.9680,  2.1924, -0.0436],
                        [-1.1008, -0.3390,  0.7427,  1.2755]])

In [7]:
%timeit t.a = new_value

77.6 ns ± 1.37 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9160, -1.1817, -0.2915],
               [ 0.9803, -0.0697,  0.6740]]),
    x: Batch(
           c: tensor([[ 0.4645, -1.2541, -0.1804, -0.9229],
                      [-1.7030, -0.9680,  2.1924, -0.0436],
                      [-1.1008, -0.3390,  0.7427,  1.2755]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9160, -1.1817, -0.2915],
        [ 0.9803, -0.0697,  0.6740]])

In [11]:
%timeit b.a

65.5 ns ± 1.23 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1437,  0.8796, -0.7154],
               [ 0.2266,  1.8378,  1.7816]]),
    x: Batch(
           c: tensor([[ 0.4645, -1.2541, -0.1804, -0.9229],
                      [-1.7030, -0.9680,  2.1924, -0.0436],
                      [-1.1008, -0.3390,  0.7427,  1.2755]]),
       ),
)

In [13]:
%timeit b.a = new_value

604 ns ± 8.34 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

986 ns ± 10.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.1 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

294 µs ± 5.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

279 µs ± 4.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4aa08046a0>
├── 'a' --> tensor([[[ 0.9160, -1.1817, -0.2915],
│                    [ 0.9803, -0.0697,  0.6740]],
│           
│                   [[ 0.9160, -1.1817, -0.2915],
│                    [ 0.9803, -0.0697,  0.6740]],
│           
│                   [[ 0.9160, -1.1817, -0.2915],
│                    [ 0.9803, -0.0697,  0.6740]],
│           
│                   [[ 0.9160, -1.1817, -0.2915],
│                    [ 0.9803, -0.0697,  0.6740]],
│           
│                   [[ 0.9160, -1.1817, -0.2915],
│                    [ 0.9803, -0.0697,  0.6740]],
│           
│                   [[ 0.9160, -1.1817, -0.2915],
│                    [ 0.9803, -0.0697,  0.6740]],
│           
│                   [[ 0.9160, -1.1817, -0.2915],
│                    [ 0.9803, -0.0697,  0.6740]],
│           
│                   [[ 0.9160, -1.1817, -0.2915],
│                    [ 0.9803, -0.0697,  0.6740]]])
└── 'x' --> <FastTreeValue 0x7f4b23ea1fa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.1 µs ± 749 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4b23ea1fd0>
├── 'a' --> tensor([[ 0.9160, -1.1817, -0.2915],
│                   [ 0.9803, -0.0697,  0.6740],
│                   [ 0.9160, -1.1817, -0.2915],
│                   [ 0.9803, -0.0697,  0.6740],
│                   [ 0.9160, -1.1817, -0.2915],
│                   [ 0.9803, -0.0697,  0.6740],
│                   [ 0.9160, -1.1817, -0.2915],
│                   [ 0.9803, -0.0697,  0.6740],
│                   [ 0.9160, -1.1817, -0.2915],
│                   [ 0.9803, -0.0697,  0.6740],
│                   [ 0.9160, -1.1817, -0.2915],
│                   [ 0.9803, -0.0697,  0.6740],
│                   [ 0.9160, -1.1817, -0.2915],
│                   [ 0.9803, -0.0697,  0.6740],
│                   [ 0.9160, -1.1817, -0.2915],
│                   [ 0.9803, -0.0697,  0.6740]])
└── 'x' --> <FastTreeValue 0x7f4a9a289280>
    └── 'c' --> tensor([[ 0.4645, -1.2541, -0.1804, -0.9229],
                        [-1.7030, -0.9680,  2.1924, -0.0436],
                 

In [23]:
%timeit t_cat(trees)

41.5 µs ± 113 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77.4 µs ± 836 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.9160, -1.1817, -0.2915],
                [ 0.9803, -0.0697,  0.6740]],
       
               [[ 0.9160, -1.1817, -0.2915],
                [ 0.9803, -0.0697,  0.6740]],
       
               [[ 0.9160, -1.1817, -0.2915],
                [ 0.9803, -0.0697,  0.6740]],
       
               [[ 0.9160, -1.1817, -0.2915],
                [ 0.9803, -0.0697,  0.6740]],
       
               [[ 0.9160, -1.1817, -0.2915],
                [ 0.9803, -0.0697,  0.6740]],
       
               [[ 0.9160, -1.1817, -0.2915],
                [ 0.9803, -0.0697,  0.6740]],
       
               [[ 0.9160, -1.1817, -0.2915],
                [ 0.9803, -0.0697,  0.6740]],
       
               [[ 0.9160, -1.1817, -0.2915],
                [ 0.9803, -0.0697,  0.6740]]]),
    x: Batch(
           c: tensor([[[ 0.4645, -1.2541, -0.1804, -0.9229],
                       [-1.7030, -0.9680,  2.1924, -0.0436],
                       [-1.1008, -0.3390,  0.7427,  1.2755]],
         

In [26]:
%timeit Batch.stack(batches)

103 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.9160, -1.1817, -0.2915],
               [ 0.9803, -0.0697,  0.6740],
               [ 0.9160, -1.1817, -0.2915],
               [ 0.9803, -0.0697,  0.6740],
               [ 0.9160, -1.1817, -0.2915],
               [ 0.9803, -0.0697,  0.6740],
               [ 0.9160, -1.1817, -0.2915],
               [ 0.9803, -0.0697,  0.6740],
               [ 0.9160, -1.1817, -0.2915],
               [ 0.9803, -0.0697,  0.6740],
               [ 0.9160, -1.1817, -0.2915],
               [ 0.9803, -0.0697,  0.6740],
               [ 0.9160, -1.1817, -0.2915],
               [ 0.9803, -0.0697,  0.6740],
               [ 0.9160, -1.1817, -0.2915],
               [ 0.9803, -0.0697,  0.6740]]),
    x: Batch(
           c: tensor([[ 0.4645, -1.2541, -0.1804, -0.9229],
                      [-1.7030, -0.9680,  2.1924, -0.0436],
                      [-1.1008, -0.3390,  0.7427,  1.2755],
                      [ 0.4645, -1.2541, -0.1804, -0.9229],
                      [-1.7030, -

In [28]:
%timeit Batch.cat(batches)

186 µs ± 2.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

554 µs ± 9.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
